<a href="https://colab.research.google.com/github/Just1got/Hello_python/blob/main/practice_seq2seq_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Translation with a Sequence to Sequence Network and Attention

Попробуем решить задачу **sequence-to-sequence**: преобразование последовательности длины $N$ в последовательность длины $T$. $T$ может быть не равно $N$.

Примеры **sequence-to-sequence** задач:
*   машинный перевод,
*   генерация ответа на вопрос,
*   генерация описания картинки или видео.

Для решения таких задач  можно использовать две **RNN**: **кодировщик** и **декодировщик**. 
* Задача **кодировщика**: обобщить информацию о **входной последовательности** $(x_1,..., x_N)$, сформировав **вектор контекста** $C$ фиксированного размера.
* Задача **декодировщика**: используя информацию из $C$, сформировать **выходную последовательность** $(y_1, ..., y_T)$.

В качестве вектора $C$ можно использовать последнее **скрытое состояние** кодировщика $h_N$.




В этом проекте будем обучать нейронную сеть переводу фраз с французского языка на английский.

::

    [KEY: > input, = target, < output]

    > il est en train de peindre un tableau .
    = he is painting a picture .
    < he is painting a picture .

    > pourquoi ne pas essayer ce vin delicieux ?
    = why not try that delicious wine ?
    < why not try that delicious wine ?

    > elle n est pas poete mais romanciere .
    = she is not a poet but a novelist .
    < she not not a poet but a novelist .

    > vous etes trop maigre .
    = you re too skinny .
    < you re all alone .



Решение задачи осуществляется на основе сети RNN вида Seq2Seq (https://arxiv.org/abs/1409.3215) Основной принцип работы которой, заключается в том, что две рекуррентные нейронные сети работают вместе для преобразования одной последовательности в другую. Кодирующая сеть сжимает входную последовательность в вектор, а сеть декодера разворачивает этот вектор в новую последовательность.

<img src ="https://edunet.kea.su/repo/EduNet-content/L08/out/seq_to_seq_with_rnn.png" width="800">


Для улучшение результатов работы используется [механизм внимания](https://arxiv.org/abs/1409.0473)_, позволяеющий декодеру
научиться фокусироваться на определенной информации входной последовательности.

Больше о Seq2Seq сетях:

-  [Learning Phrase Representations using RNN Encoder-Decoder for
   Statistical Machine Translation](https://arxiv.org/abs/1406.1078)_
-  [Sequence to Sequence Learning with Neural
   Networks](https://arxiv.org/abs/1409.3215)_
-  [Neural Machine Translation by Jointly Learning to Align and
   Translate](https://arxiv.org/abs/1409.0473)_
-  [A Neural Conversational Model](https://arxiv.org/abs/1506.05869)_


### Проблемы Sequence-to-Sequence with RNNs

При этом возникают проблемы:
- **Вектор контекста** $c$ фиксированной длины не может вместить любое количество информации, поэтому для длинных последовательностей качество будет ухудшаться. 
- На каждой итерации декодировщика **скрытое состояние** $s_t$ должно сохранять информацию о том, какие элементы **выходной последовательности** уже были сгенерированы. Если $s_t$ не способно вместить эту информацию, модель может зациклиться или потерять часть **выходной последовательности**.

**Вектор контекста** $c$ и **скрытые состояния** декодировщика $s_t$ являются “бутылочными горлышками” модели.

## Sequence-to-Sequence with RNNs and Attention mechanism

Как решить проблему “бутылочного горлышка”?
* формировать свой контекст $c_t$ для каждого элемента **выходной последовательности** $y_t$,  
* использовать для формирования контекста $c_t$ все **скрытые состояния** кодировщика $h_i$.

Для формирования **векторов контекста** $(c_1, ..., c_T)$ возьмем линейную комбинацию **скрытых состояний** кодировщика $h_i$ с весами $a_{ti}$:
$$ c_t=\sum_{i=1}^{N}a_{ti}h_i.$$

Веса $a_{ti}$ указывают, какие **скрытые состояния** кодировщика $h_i$ важны для формирования элемента **выходной последовательности** $y_t$.  Они “показывают” декодировщику куда “смотреть” при генерации данного элемента. Такой механизм в нейросетях получил название **attention** (внимание).

Веса $a_{ti}$ предсказывает сама модель. Для удобства веса подбираются таким образом, чтобы их сумма для каждого **вектора контекста** $c_t$ была равна 1 (нормализация):
$$ \sum_{i=1}^{N}a_{ti} = 1,$$

$$  0\leqslant a_{ti} \leqslant 1.$$

Для этого на выходе предсказывающего веса слоя ставят **SoftMax**.

Чтобы **вектор контекста** $c_t$ содержал информацию о уже сгенеренных элементах **выходной последовательности**, значение веса до нормализации  $e_{ti}$ зависит не только от скрытого состояния кодировщика $h_i$, но и от предыдущего скрытого состояния декодировщика $s_{t-1}$.

**Модель внимания — сходство входного и выходного состояния.**

$a(h, h^{'})$ — функция сходства состояний входа $h$ и выхода $h^{'}$ 

$a_{ti}$  — важность входа $i$ для выхода $t$ (attention score), $ \sum_{i=1}a_{ti} = 1$

$c_t$ — вектор входного контекста для выхода t (context vector)

<img src ="https://edunet.kea.su/repo/EduNet-content/redraw/L08_att_layer_rnn.png" width="800">

$h_i = f_{in} (h_i, h_{i-1})$

$a_{ti} = norm_i a(h_i, h_{t-1}^{'})$

$c_{t} = \sum_{i=1} А_{ti}h_i$

$h_t^{'} = f_{out}(h_{t-1}^{'}, y_{t-1}, c_t)$

$y_t = f_y(h_{t}^{'}, y_{t-1}, c_t)$


*   Обучаемые параметры в $А$ и $с$
*   Можно отказаться от реккурентности на входе

$norm_i(p_i) = p_i / \sum_{k}p_{k}$ 


Чтобы сгенерировать элемент выходной последовательности $h_t$, у нас уже сгенерированы все предыдущие $h_{t-1}, h_{t-2}...$. Смотрим на них и на всю последовательность входа $h_{i}$.

Оператор нормировки даёт нам вероятностную модель, на сколько элемент $h_{i}$ полезен для синтеза $h_{t}$.

$c_{t}$ — вектор контекста для $h_{t}$, состоящий из взвешенных элементов $h_{i}$

## Проблема attention

Attention решает проблему "забывания" при работе с последовательностями. Но цена этого решения — квадратичное возрастание вычислительной сложности с ростом длины последовательности.

Вычислительная сложность **одного слоя RNN** составляет $O(bn d^2)$, где $b$ — длина батча, $n$ — число токенов и $d$ — размерность входа. Часть $d^2$ обусловлена матричным перемножением внутри блока RNN.

Вычислительная сложность **одного слоя attention** в простейшей реализации составляет $O(bn^2 d)$, то есть растет квадратично при росте длины последовательности $n$. Это объясняется тем, что длина выходной последовательности приблизительно равна длине входной последовательности $n$, и необходимо для каждого выходного токена рассчитать коэффициенты attention со всеми входными токенами. Сложность расчета одного коэффициента в простейшем случае составляет $O(d)$.

Ни рекуррентные сети, ни attention не могут эффективно работать с очень длинными последовательностями. RNN/LSTM "забывают" начало последовательности, а attention просто не может выполнить расчет за разумное время.

На практике attention предпочтительнее, потому что удобнее иметь модель, которая или работает адекватно или не работает вообще, чем модель, которая работает неадекватно ("забывает" контекст) без предупреждения.

Обычно для attention используют достаточно большую длину последовательности, чтобы в нее могло поместиться практически любое предложение или даже несколько предложений, например, 512 токенов.

# Loading data files

В качестве входных данных используется набор пар-переводов с французского на английский.

Пары Английский - Французский содержатся отдельным файлом ``data/eng-fra.txt``. В файле содержатся пары вида:

::

    I am cold.    J'ai froid.


   [Скачать](https://download.pytorch.org/tutorial/data.zip)
 



In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/My\ Drive

/content/drive/My Drive


В данном случае будем использовать one=hot-encoding, для представления слов.


Объявим вспомогательный класс ``Lang``, который будет последовательно представлять word → index (``word2index``) и index → word
(``index2word``), а также счётчик использования слов ``word2count``, который будет использоваться для замены редких слов в дальнейшем.


SOS - start of sequence token

EOS - end of sequence token




In [ ]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

Все файлы представлены в формате Unicode, для упрощения мы преобразуем символы Unicode в ASCII, сделаем все буквы строчными и уберем большинство знаков препинания.




In [ ]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

Для чтения файла данных мы разобьем файл на строки, а затем разобьем строки на пары. Файлы представлены English → Other Language, поэтому, если мы хотим изменить последовательность Other Language → English в методе определено условие ``reverse``.




In [ ]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

Поскольку имеется *множество* примеров предложений, и мы хотим обучить быстро, мы сократим набор данных только до относительно коротких и простые предложения. Здесь максимальная длина составляет 10 слов (включая конечную пунктуацию), также отфильтровываем предложения, которые переводятся в форму "I am", "He is" и тд (с учетом апострофов, замененных ранее).




In [ ]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

Полный процесс подготовки данных выглядит следующим образом:

-  Чтение текстового файла и разбиение на строки, разбиение строк на пары
-  Нормализация текста, фильтрация по длине и содержанию
-  Составить списки слов из предложений в парах




In [ ]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Counting words...
Counted words:
fra 4345
eng 2803
['je suis sur le point de quitter ici .', 'i am about to leave here .']


## The Seq2Seq Model

В отличие от предсказания последовательности с помощью одной RNN, где каждому входу последующей ячейки  соответствует выход предыдущей, модель seq2seq освобождает нас от последовательного представления, что делает ее идеальной для перевода между двумя языками.

Рассмотрим предложение ``Je ne suis pas le chat noir`` → ``I am not the
black cat``. Большинство слов во входном предложении имеют прямой перевод в выходном предложении, но в несколько ином порядке порядке: ``chat noir`` and ``black cat``. 
Также во французском варианте присутствуют артикли ``ne/pas``, что отличает длину входной последовательности от выходной. Было бы трудно создать правильный перевод непосредственно из последовательности входных слов.

В модели seq2seq кодер создает единый вектор, который в идеальном случае кодирует "смысл" входной последовательности в единый вектор - единственную точку в некотором N-мерном пространстве предложений.




### The Encoder

Кодер сети seq2seq - это RNN, которая выдает некоторое значение для каждого слова из входного предложения. Для каждого входного слова кодер выдает вектор и скрытое состояние, и использует скрытое состояние для следующего входного слова.







In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

### The Decoder

Декодер - это другая RNN, которая принимает выходной вектор(ы) кодера и выдает последовательность слов для создания перевода.




#### Simple Decoder

В простейшем декодере seq2seq мы используем только последний выход кодера.
Этот последний выход иногда называют *вектором контекста*, так как он кодирует контекст всей последовательности. Этот вектор контекста используется в качестве начального скрытого состояния декодера.

На каждом этапе декодирования декодеру дается входной токен и скрытое состояние. Начальным входным маркером является маркер начала строки ``<SOS>'', а первым скрытым состоянием - вектор контекста (последнее скрытое состояние кодера).




In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

#### Attention Decoder

Если между кодером и декодером передается только одним вектором контекста, это может привести к потере начальной информации.

Механизм Внимания позволяет сети декодера "фокусироваться" на различной части выходов кодера для каждого шага собственных выходов декодера. Сначала мы вычисляем набор *весов внимания*. Они будут умножены на выходные векторы кодера для создания взвешенной комбинации. Результат (называемый в коде ``attn_applied``) должен содержать информацию о конкретной части входной последовательности и, таким образом, помочь декодеру выбрать правильные выходные слова.

Вычисление весов внимания осуществляется с помощью другого слоя с прямой связью ``attn``, использующего в качестве входных данных вход декодера и скрытое состояние. Поскольку в обучающих данных присутствуют предложения всех размеров, для создания и обучения этого слоя необходимо выбрать максимальную длину предложения (длина входных данных, для выходов кодера), к которой он может применяться. Предложения максимальной длины будут использовать все веса внимания, в то время как более короткие предложения будут использовать только первые несколько.







In [ ]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)



# Training

### Preparing Training Data

Для обучения, для каждой пары нам понадобится входной тензор (индексы слов во входном предложении) и целевой тензор (индексы слов во входном предложении). слов во входном предложении) и целевой тензор (индексы слов в целевом предложении). При создании этих векторов мы будем добавлять EOS токен к обеим последовательностям.




In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

### Training the Model

Для обучения мы пропускаем входное предложение через кодировщик и отслеживаем каждый выход и последнее скрытое состояние. Затем декодеру дается лексема <SOS> в качестве первого входа и последнее скрытое состояние кодера в качестве первого скрытого состояния.




In [ ]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

Это вспомогательная функция для печати прошедшего времени и предполагаемого времени окончания  и % выполнения обучения.




In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

Процесс обучения выглядит следующим образом:

-  Начало последовательности
-  Инициализация оптимизаторв
-  Создать обучающие пары
-  Запуск пустого массива потерь для построения графика



In [ ]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

### Plotting results






In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

## Evaluation

Происходит в основном так же, как и обучение, но здесь нет целей, поэтому на каждом шаге мы просто возвращаем предсказания декодера обратно самому себе. Каждый раз, когда он предсказывает слово, мы добавляем его в выходную строку, и если он предсказывает лексему EOS, мы останавливаемся на этом. Мы также сохраняем результаты внимания декодера для последующего отображения.



In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

Мы можем оценить случайные предложения из обучающего набора и вывести входные, целевые и выходные данные, чтобы сделать некоторые оценки качества:




In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

## Training and Evaluating






In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 75000, print_every=5000)

4m 37s (- 64m 46s) (5000 6%) 2.8239
9m 47s (- 63m 40s) (10000 13%) 2.2907


KeyboardInterrupt: ignored

In [ ]:
evaluateRandomly(encoder1, attn_decoder1)

> je ne suis plus un bebe !
= i m not a baby you know !
< i m not a ! <EOS>

> vous n etes pas contusionnees .
= you re not bruised .
< you re not a . . <EOS>

> tu es probablement trop jeune pour le comprendre .
= you re probably too young to understand this .
< you are too young too for for the . <EOS>

> vous n etes pas une sainte .
= you re no saint .
< you aren t a . <EOS>

> nous sommes veinards .
= we re in luck .
< we re different . <EOS>

> je ne fais pas partie de leur bande .
= i m not one of them .
< i m not your of . . <EOS>

> il est malpoli .
= he is a rude person .
< he is a . <EOS>

> ils sont partis .
= they re gone .
< they re different . <EOS>

> nous travaillons aussi vite que nous le pouvons .
= we re working as fast as we can .
< we re going to go all . <EOS>

> j ai la tete qui me tourne .
= i m feeling dizzy .
< i m feeling . . <EOS>

